In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [9]:
from selenium import webdriver
import time

driver = webdriver.Chrome()
driver.get("http://www.google.com/")
search_box = driver.find_element_by_name("q")
search_box.send_keys("ChromeDriver")
search_box.submit()
time.sleep(5)
driver.quit()

<IPython.core.display.Javascript object>

In [10]:
import cv2
import numpy as np
import imutils
import pickle


def read_CAPTCHA_image(captcha_image_file):
    """Read CAPTCHA image into numpy array using OpenCv."""
    return cv2.imread(captcha_image_file)


def grayscale_CAPTCHA_image(captcha_image):
    "Grayscales CAPTCHA image." ""
    return cv2.cvtColor(captcha_image, cv2.COLOR_BGR2GRAY)


def threshold_CAPTCHA_image(captcha_image_grayscaled):
    """Thresholds CAPTCHA image."""
    return cv2.threshold(
        captcha_image_grayscaled, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU
    )[1]


def dilate_characters(binary_image):
    """slightly expands the characters."""
    kernel = np.ones((2, 2), np.uint8)
    return cv2.dilate(binary_image, kernel, iterations=1)


def find_CAPTCHA_contours(captcha_image_thresholded):
    """Compute the contours of characters in the CAPTCHA image."""
    return cv2.findContours(
        captcha_image_thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )[0]


def compute_bounding_rectangles(contours):
    """Computes the bounding rectangles of the contours."""
    return list(map(cv2.boundingRect, contours))


def split_fat_rectangles(rectangles):
    """Splits fat rectangles into two rectangles."""
    letter_bounding_rectangles = []
    for rectangle in rectangles:
        (x, y, w, h) = rectangle
        if w / h > 1.25:
            half_width = int(w / 2)
            letter_bounding_rectangles.append((x, y, half_width, h))
            letter_bounding_rectangles.append((x + half_width, y, half_width, h))
        else:
            letter_bounding_rectangles.append(rectangle)
    return letter_bounding_rectangles


def get_character_images(rectangles, image):
    """Extracts the characters defined by bounding rectangles."""
    char_images = []
    for rect in rectangles:
        x, y, w, h = rect
        char_image = image[y - 1 : y + h + 1, x - 1 : x + w + 1]
        char_images.append(char_image)
    return char_images


def sort_bounding_rectangles(rects):
    """Sorts bounding rectangles by x coordinate."""
    return sorted(rects, key=lambda x: float(x[0]))


def grayscale(image):
    """Grayscales image using openCV."""
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


def normalize_dimensions(image, desired_width=20, desired_height=20):
    """Resizes image to desired dimensions using white padding and rescaling."""
    (h, w) = image.shape[:2]
    if w > h:
        image = imutils.resize(image, width=desired_width)
    else:
        image = imutils.resize(image, height=desired_height)
    width_padding = int((desired_width - image.shape[1]) / 2)
    height_padding = int((desired_height - image.shape[0]) / 2)
    WHITE = [255, 255, 255]
    image_with_border = cv2.copyMakeBorder(
        image,
        height_padding,
        height_padding,
        width_padding,
        width_padding,
        cv2.BORDER_CONSTANT,
        value=WHITE,
    )
    image_with_border_resized = cv2.resize(
        image_with_border, (desired_width, desired_height), interpolation=cv2.INTER_AREA
    )
    return image_with_border_resized


def reshape_for_keras(image):
    """Adds a dummy dimension to fit keras's input requirements."""
    return np.expand_dims(image, axis=2)


def CAPTCHA_to_characters_pipeline(CAPTCHA):
    """Crop the character of a CAPTCHA."""
    captcha_image = read_CAPTCHA_image(CAPTCHA)
    captcha_image_grayscaled = grayscale_CAPTCHA_image(captcha_image)
    captcha_image_thresholded = threshold_CAPTCHA_image(captcha_image_grayscaled)
    captcha_image_dilated = dilate_characters(captcha_image_thresholded)
    captcha_image_contours = find_CAPTCHA_contours(captcha_image_dilated)
    character_bounding_rectangles = split_fat_rectangles(
        compute_bounding_rectangles(captcha_image_contours)
    )
    character_bounding_rectangles = sort_bounding_rectangles(
        character_bounding_rectangles
    )
    character_images = get_character_images(
        character_bounding_rectangles, captcha_image
    )
    return character_images

<IPython.core.display.Javascript object>

In [11]:
def load_classifier():
    """Load the pretrained CNN."""
    from keras.models import Sequential
    from keras.layers.convolutional import Conv2D, MaxPooling2D
    from keras.layers.core import Flatten, Dense
    import cv2
    import imutils
    import pickle
    import numpy as np

    num_classes = 32
    CNN_model = Sequential()
    CNN_model.add(
        Conv2D(20, (5, 5), padding="same", input_shape=(20, 20, 1), activation="relu")
    )
    CNN_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    CNN_model.add(Conv2D(50, (5, 5), padding="same", activation="relu"))
    CNN_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    CNN_model.add(Flatten())
    CNN_model.add(Dense(512, activation="relu"))
    CNN_model.add(Dense(num_classes, activation="softmax"))
    CNN_model.compile(
        loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    CNN_model.load_weights("weights.h5")
    return CNN_model

<IPython.core.display.Javascript object>

In [12]:
def load_label_binarizer():
    """Load the label binarizer."""
    return pickle.load(open("binarizer.pkl", "rb"))

<IPython.core.display.Javascript object>

In [13]:
def load_model():
    """Loads the CNN model and label binarizer."""
    CNN_model = load_classifier()
    label_binarizer = load_label_binarizer()
    return CNN_model, label_binarizer

<IPython.core.display.Javascript object>

In [14]:
def predict_on_characters(character_images, model, label_binarizer):
    """Predicts each character and then outputs the resulting prediction."""
    X = []
    for image in character_images:
        image_gray = grayscale(image)
        image_normalized = normalize_dimensions(image_gray)
        image_reshaped_for_keras = reshape_for_keras(image_normalized)
        X.append(image_reshaped_for_keras)

    X = np.array(X, dtype="float") / 255.0
    pred = model.predict(X)
    predicted_text = label_binarizer.inverse_transform(pred)
    return predicted_text

<IPython.core.display.Javascript object>

In [15]:
model, label_binarizer = load_model()

Using TensorFlow backend.


<IPython.core.display.Javascript object>

In [16]:
import urllib.request

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
driver.get(
    "https://ml4cs.com/CAPTCHA/index.php/break-me-use-a-bot-to-crack-the-captcha/"
)
time.sleep(2)
captcha_image = driver.find_element_by_css_selector(".wpcf7-captcha-captcha-170")
src = captcha_image.get_attribute("src")
urllib.request.urlretrieve(src, "captcha.png")
pred = predict_on_characters(
    CAPTCHA_to_characters_pipeline("captcha.png"), model, label_binarizer
)
captcha_input = driver.find_element_by_name("captcha-170")
captcha_input.send_keys(pred)
captcha_input.submit()
time.sleep(5)
driver.quit()

<IPython.core.display.Javascript object>